In [1]:
import pickle
import torch
# Load the pickle file
with open('processed_image_features.pkl', 'rb') as f:
    img_data = pickle.load(f)
    
# Load the pickle file
with open('text_features.pkl', 'rb') as f:
    text_data = pickle.load(f)

In [2]:
img_features = img_data['image_feature']
img_files = img_data['files']
text_features = text_data['text_features']

print(img_features.shape)
print(text_features.shape)



torch.Size([10000, 512])
torch.Size([6, 512])


In [3]:
# Compute cosine similarity
similarity_matrix = torch.matmul(img_features, text_features.T)

In [4]:
# Accuracy calculate 
import pandas as pd 
df = pd.read_csv('clean_dataset.csv')
df = df[['filename', 'quantity']]
data_dict = df.set_index('filename')['quantity'].to_dict()
acc = 0
for index, filename in enumerate(img_files):
    if filename in data_dict:
        ground_truth = data_dict[filename]
        max_similarity_score, max_text_index = torch.max(similarity_matrix[index, :], dim=0)
        if max_text_index == ground_truth:
            acc +=1 
    else:
        pass

In [5]:
(acc / 10000)

0.2138

In [6]:
similarity_matrix_text = torch.matmul(text_features, text_features.T)

In [7]:
similarity_matrix_text

tensor([[1.0000, 0.9975, 0.9936, 0.9937, 0.9925, 0.9912],
        [0.9975, 1.0000, 0.9978, 0.9975, 0.9963, 0.9944],
        [0.9936, 0.9978, 1.0000, 0.9977, 0.9950, 0.9905],
        [0.9937, 0.9975, 0.9977, 1.0000, 0.9989, 0.9953],
        [0.9925, 0.9963, 0.9950, 0.9989, 1.0000, 0.9981],
        [0.9912, 0.9944, 0.9905, 0.9953, 0.9981, 1.0000]])

In [8]:
# Testing image embeddings quality 
from PIL import Image

import os
import clip
import torch
from PIL import Image

# Load the model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)
path = "bin_images"
image_data = Image.open(path + "/" + img_files[200])
batch_image_input = preprocess(image_data).unsqueeze(0).to(device)
with torch.no_grad():
    image_features = model.encode_image(batch_image_input)    
    
image_features /= image_features.norm(dim=-1, keepdim=True)

In [10]:
print(f"Baseline accuracy among 6 classes = {1/6}")

Baseline accuracy among 6 classes = 0.16666666666666666
